In [3]:
from pyspark.sql.types import StructType
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from pyspark.sql.streaming import DataStreamWriter
from minio import Minio
from delta.tables import *
import os

def minio_session_spark():
    spark = (
        SparkSession.builder
            .master("local[*]")
            .appName("appMinIO")
            ### Config Fields
            .config('spark.sql.debug.maxToStringFields', 5000)
            .config('spark.debug.maxToStringFields', 5000)
            ### Optimize
            .config("delta.autoOptimize.optimizeWrite", "true")
            .config("delta.autoOptimize.autoCompact", "true")
            ### Delta Table
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            ## MinIO
            #.config("spark.hadoop.fs.s3a.endpoint", "http://172.20.0.2:9000")
             .config("spark.hadoop.fs.s3a.endpoint", "minio:9000")

            .config("spark.hadoop.fs.s3a.access.key", "tcc_user")
            .config("spark.hadoop.fs.s3a.secret.key", "Acnmne@a9h!")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            ## Jars
            .config("spark.jars", "/home/jovyan/work/jars/hadoop-common-3.3.2.jar,\
                                    /home/jovyan/work/jars/hadoop-aws-3.3.2.jar, \
                                    /home/jovyan/work/jars/aws-java-sdk-bundle-1.11.874.jar")
            .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
            .getOrCreate()
    )
    return spark

In [4]:
spark = minio_session_spark()

# spark
print(f"Spark version = {spark.version}")

# hadoop
print(f"Hadoop version = {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Spark version = 3.3.2
Hadoop version = 3.3.2


In [ ]:
# Configure Minio connection
minio_endpoint = 'minio:9000'
access_key = 'tcc_fia'
secret_key = 'tcc_fia'
secure = False  # Set to True for HTTPS
minio_client = Minio(endpoint=minio_endpoint, access_key=access_key, secret_key=secret_key, secure=secure)

# Specify the Minio bucket and path
minio_bucket = 'raw'
minio_path_bronze_players = ['bronze_I/', 'bronze_II/','bronze_III/','bronze_IV/']
minio_path_silver_players = ['silver_I/', 'silver_II/','silver_III/','silver_IV/']
minio_path_ouro_players = ['gold_I/', 'gold_II/','gold_III/','gold_IV/']
minio_path_platina_players = ['platinum_I/', 'platinum_II/','platinum_III/','platinum_IV/']
# minio_path_esmeralda_players = ['emerald_I/', 'emerald_II/','emerald_III/','emerald_IV/']
# minio_path_diamante_players = ['diamond_I/', 'diamond_II/','diamond_III/','diamond_IV/']


#BRONZE PLAYERS
for rank in minio_path_bronze_players:
    print(f'Executando {rank}')
    objects = minio_client.list_objects(minio_bucket, prefix=rank)

    # Read JSON files into a PySpark DataFrame
    json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
    df = spark.read.json(json_files[1:])
    
    #Saving Customer Segmentation in Bronze Layer
    (
        df
        .write
        .format("delta")
        .mode("overwrite") 
        .option("overwriteSchema", "True")
        .save(f"s3a://bronze/" + f'{rank}')
    )

#SILVER PLAYERS
for rank in minio_path_silver_players:
    print(f'Executando {rank}')
    objects = minio_client.list_objects(minio_bucket, prefix=rank)

    # Read JSON files into a PySpark DataFrame
    json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
    df = spark.read.json(json_files[1:])
    
    #Saving Customer Segmentation in Bronze Layer
    (
        df
        .write
        .format("delta")
        .mode("overwrite") 
        .option("overwriteSchema", "True")
        .save(f"s3a://bronze/" + f'{rank}')
    )
    
# #GOLD PLAYERS
# for rank in minio_path_ouro_players:
#     objects = minio_client.list_objects(minio_bucket, prefix=rank)

#     # Read JSON files into a PySpark DataFrame
#     json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
#     df = spark.read.json(json_files[1:])
    
#     #Saving Customer Segmentation in Bronze Layer
#     (
#         df
#         .write
#         .format("delta")
#         .mode("overwrite") 
#         .option("overwriteSchema", "True")
#         .save(f"s3a://bronze/" + f'{rank}')
#     )

# #PLATINA PLAYERS
# for rank in minio_path_platina_players:
#     objects = minio_client.list_objects(minio_bucket, prefix=rank)

#     # Read JSON files into a PySpark DataFrame
#     json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
#     df = spark.read.json(json_files[1:])
    
#     #Saving Customer Segmentation in Bronze Layer
#     (
#         df
#         .write
#         .format("delta")
#         .mode("overwrite") 
#         .option("overwriteSchema", "True")
#         .save(f"s3a://bronze/" + f'{rank}')
#     )
    
# #ESMERALDA PLAYERS
# for rank in minio_path_esmeralda_players:
#     objects = minio_client.list_objects(minio_bucket, prefix=rank)

#     # Read JSON files into a PySpark DataFrame
#     json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
#     df = spark.read.json(json_files[1:])
    
#     #Saving Customer Segmentation in Bronze Layer
#     (
#         df
#         .write
#         .format("delta")
#         .mode("overwrite") 
#         .option("overwriteSchema", "True")
#         .save(f"s3a://bronze/" + f'{rank}')
#     )
    
# #DIAMANTE PLAYERS
# for rank in minio_path_diamante_players:
#     objects = minio_client.list_objects(minio_bucket, prefix=rank)

#     # Read JSON files into a PySpark DataFrame
#     json_files = [f"s3a://{minio_bucket}/{obj.object_name}" for obj in objects]
#     df = spark.read.json(json_files[1:])
    
#     #Saving Customer Segmentation in Bronze Layer
#     (
#         df
#         .write
#         .format("delta")
#         .mode("overwrite") 
#         .option("overwriteSchema", "True")
#         .save(f"s3a://bronze/" + f'{rank}')
#     )